<a href="https://colab.research.google.com/github/chsr00/AI_FOR_CAE2/blob/main/CAE_for_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, UpSampling2D, Input, Convolution2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

import scipy.io
from sklearn.model_selection import train_test_split

print(tf.__version__)

In [ ]:
from google.colab import files

uploaded = files.upload()

# 코드가 돌아가면 파일선택 버튼을 눌러서 upload할 파일 선택
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
from google.colab import drive
# google drive와 연결 (인증코드 입력해야함)
drive.mount('/gdrive')

# data_3000 폴더 만듦. zip 파일 압출 풀기
! mkdir data_3000
! unzip data_3000.zip -d ./data_3000

In [ ]:
# 압축해제된 데이터 경로 (데이터 있는 파일 경로 입력)
src = './data_3000/'

# 이미지 읽기 및 출력
def img_read_plot(src,file):
    img = cv.imread(src+file,cv.COLOR_BGR2GRAY)
    plt.imshow(img)
    plt.xticks([]) # x축 눈금
    plt.yticks([]) # y축 눈금
    plt.show()
    return img

#이미지 읽기
def img_read(src,file):
    img = cv.imread(src+file,cv.COLOR_BGR2GRAY)
    return img

#src 경로에 있는 파일 명을 저장
files = os.listdir(src)

X,Y = [],[]
count = 0

# 경로와 파일명을 입력으로 넣어 확인하고
# 데이터를 255로 나눠서 0~1사이로 정규화 하여 X 리스트 추가

for file in files:
  # 데이터의 일부분만 확인해봅니다.
  if count < 10 :
    print(count)
    X.append(img_read_plot(src,file)/255.)
    Y.append(float(file[:-4]))
    count+=1
  else :
    X.append(img_read(src,file)/255.)
    Y.append(float(file[:-4]))

# 전체 X, Y data shape 출력
print('X_shape:',np.shape(X[0]),'Y_shape:',np.shape(Y[0]))
print('X_list shape:',np.shape(X),'Y_list shape:',np.shape(Y))

# hyper parameter 설정
img_size = 56       # 이미지 사이즈
latent_dim = 32     # latent dimension 크기 설정
BUFFER_SIZE = 5600  # 총 이미지 갯수
BATCH_SIZE = 20     # 배치 사이즈(나눴을때 이미지 갯수에 딱 떨어지게 설정해야 함)

# Train set(80%), Test set(20%)으로 나누기
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=1,shuffle=True)

# CNN layer에 들어갈 수 있게 (x, 56, 56, 1) 차원으로 맞춰줌
X_train = np.reshape(X_train, (len(X_train), img_size, img_size, 1))
X_test = np.reshape(X_test, (len(X_test), img_size, img_size, 1))
print(np.shape(X_train),np.shape(X_test))

In [ ]:
# 인코더
def encoder():

  model = tf.keras.Sequential()
  model.add(Conv2D(5, (3, 3), activation='relu', padding='same', input_shape=(img_size, img_size, 1)))
  model.add(MaxPooling2D((2, 2), padding='same'))
  model.add(Conv2D(10, (3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2), padding='same'))
  model.add(Conv2D(15, (3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2), padding='same'))
  model.add(Conv2D(20, (3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2), padding='same'))
  model.add(Conv2D(25, (3, 3), activation='relu', padding='same'))
  model.add(Flatten())
  # latent dimension으로 축소됨
  model.add(Dense(latent_dim))

  return model

In [ ]:
# 인코더 함수를 실행해서 모델을 build 후 변수에 집어넣고, 모델 구조를 살펴봄
e_model = encoder()
e_model.summary()
plot_model(e_model, show_shapes=True)

In [ ]:
# 디코더
def decoder():

  model = tf.keras.Sequential()
  model.add(Dense(7 * 7 * 56, input_shape=(latent_dim,)))
  model.add(Reshape((7, 7, 56)))
  model.add(Conv2D(56, (2, 2), activation='relu', padding='same'))
  model.add(UpSampling2D((2, 2)))
  model.add(Conv2D(56, (2, 2), activation='relu', padding='same'))
  model.add(UpSampling2D((2, 2)))
  model.add(Conv2D(56, (2, 2), activation='relu', padding='same'))
  model.add(UpSampling2D((2, 2)))
  model.add(Dropout(0.5))
  model.add(Conv2D(1, (2, 2), padding='same'))

  return model

In [ ]:
# 디코더 함수를 실행해서 모델을 build 후 변수에 집어넣고, 모델 구조를 살펴봄
d_model = decoder()
d_model.summary()
plot_model(d_model, show_shapes=True)

In [ ]:
# 모델을 합쳐줌
input_img = Input(shape=(img_size, img_size, 1))
model = Model(input_img, d_model(e_model(input_img)), name='autoencoder')

# optimizer 설정과 model complie하면서 optimizer와 loss 함수, loss 평가지표로 mae 설정
adam=Adam(lr=0.00008, beta_1=0.9)
model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mae'])

In [ ]:
# model.fit을 써서 학습. 학습과정을 history에 저장
history = model.fit(X_train, X_train,
                    batch_size=20,
                    epochs=100,
                    verbose=2,
                    validation_data=(X_test, X_test),
                    shuffle=True)

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('train loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for mean_absolute_error
plt.plot(history.history['val_loss'])
plt.title('test loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
sample_size = 10                        # 보여줄 sample 갯수 설정
prediction = model.predict(X_test)      # test 데이터로 이미지 복원(predict 함수 사용)

# plot 하기
fig, ax = plt.subplots(2, sample_size, figsize=(15, 4))
for i in range(sample_size):
  org_img = X_test[i].reshape(56,56)
  rec_img = prediction[i].reshape(56,56)

  ax[0][i].set_axis_off()
  ax[1][i].set_axis_off()

  ax[0][i].imshow(org_img, cmap=plt.cm.bone)
  ax[1][i].imshow(rec_img, cmap=plt.cm.bone)

plt.show()